In [1]:
#if run on local server, the installation of pyreadr, numpy, matplotlib, pandas, geopandas, reverse_geocoder and bokeh is needed.
import pyreadr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import reverse_geocoder as rg #should use pip to install

import json

import math

import bokeh
from bokeh.io import show
from bokeh.models import (ColorBar, ColumnDataSource,
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, Column,Legend,LegendItem)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.models import Range1d

from ipywidgets import interact, FloatSlider
from bokeh.io import output_notebook, show, push_notebook

output_notebook(hide_banner=True)

In [2]:
#import files : the dataset and the shapefile for the map
result = pyreadr.read_r('../Data/data_train_DF.RData') 
df1 = result["data_train_DF"]
df1['BA'] = df1['BA'].fillna(0)
df1['CNT'] = df1['CNT'].fillna(0)

#read shp file containing polygons of the US states 
#source: https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
states1=gpd.read_file("../Data/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")

#extract example data
wf=df1[df1['year']==1993]
wf=wf[wf['month']==3]

In [3]:
#search where the coordinates are with reverse_geocoder, if not in the U.S. include in the closest U.S state
l=[]
for i in range(len(wf)):
    p=(wf.lat.iloc[i],wf.lon.iloc[i])
    l.append(p)
coor=l
res=rg.search(coor)#search where the location belongs to
for i in res:
    if i["admin1"]=='Tamaulipas':
        i["admin1"]='Texas'
    if i["admin1"]=='British Columbia':
        i["admin1"]='Washington'
    if i["admin1"]=='Ontario':
        i["admin1"]="Minnesota"
    if i["admin1"]=='Chihuahua':
        i["admin1"]="New Mexico"
    if i["admin1"]=='Coahuila':
        i["admin1"]="Texas"
    if i["admin1"]=='New Brunswick':
        i["admin1"]="Maine"
    if i["admin1"]=='Sonora':
        i["admin1"]='Arizona'
    if i["admin1"]=='Baja California':
        i["admin1"]='California'
    if i["admin1"]=='Quebec':
        i["admin1"]="New York"

Loading formatted geocoded file...


In [4]:
#useful dictionaries

#total area of the states
#source from https://www.census.gov/geographies/reference-files/2010/geo/state-area.html
totarea={
    "Alabama": 135767,
    "Arizona": 295234,
    "Arkansas": 137732,
    "California": 423967,
    "Colorado": 269601,
    "Connecticut": 14357,
    "Delaware": 6446,
    "Florida": 170312,
    "Georgia": 153910,
    "Idaho": 216443,
    "Illinois": 149995,
    "Indiana": 94326,
    "Iowa": 145746,
    "Kansas": 213100,
    "Kentucky": 104656,
    "Louisiana": 135659,
    "Maine": 91633,
    "Maryland": 32131,
    "Massachusetts": 27336,
    "Michigan": 250487,
    "Minnesota": 225163,
    "Mississippi": 125438,
    "Missouri": 180540,
    "Montana": 380831,
    "Nebraska": 200330,
    "Nevada": 286380,
    "New Hampshire": 24214,
    "New Jersey": 22591,
    "New Mexico": 314917,
    "New York": 141297,
    "North Carolina": 139391,
    "North Dakota": 183108,
    "Ohio": 116098,
    "Oklahoma": 181037,
    "Oregon": 254799,
    "Pennsylvania": 119280,
    "Rhode Island": 4001,
    "South Carolina": 82933,
    "South Dakota": 199729,
    "Tennessee": 109153,
    "Texas": 695662,
    "Utah": 219882,
    "Vermont": 24906,
    "Virginia": 110787,
    "Washington": 184661,
    "West Virginia": 62756,
    "Wisconsin": 169635,
    "Wyoming": 253335}

In [5]:
#initialize the counts for each state
def count_init():
    d={
    "Alabama": 0,
    "Alaska": 0,
    "Arizona": 0,
    "Arkansas": 0,
    "California": 0,
    "Colorado": 0,
    "Connecticut": 0,
    "District of Columbia": 0,
    "Delaware": 0,
    "Florida": 0,
    "Georgia": 0,
    "Hawaii": 0,
    "Idaho": 0,
    "Illinois": 0,
    "Indiana": 0,
    "Iowa": 0,
    "Kansas": 0,
    "Kentucky": 0,
    "Louisiana": 0,
    "Maine": 0,
    "Maryland": 0,
    "Massachusetts": 0,
    "Michigan": 0,
    "Minnesota": 0,
    "Mississippi": 0,
    "Missouri": 0,
    "Montana": 0,
    "Nebraska": 0,
    "Nevada": 0,
    "New Hampshire": 0,
    "New Jersey": 0,
    "New Mexico": 0,
    "New York": 0,
    "North Carolina": 0,
    "North Dakota": 0,
    "Ohio": 0,
    "Oklahoma": 0,
    "Oregon": 0,
    "Pennsylvania": 0,
    "Rhode Island": 0,
    "South Carolina": 0,
    "South Dakota": 0,
    "Tennessee": 0,
    "Texas": 0,
    "Utah": 0,
    "Vermont": 0,
    "Virginia": 0,
    "Washington": 0,
    "West Virginia": 0,
    "Wisconsin": 0,
    "Wyoming": 0}
    return d

#functions to update the dictionary

def countfire(res,df):#for CNT
    stcount=count_init()
    for i in range(len(res)):
        if res[i]['admin1'] in totarea.keys():
            stcount[res[i]['admin1']]+=df.CNT.iloc[i]/totarea[res[i]['admin1']]*100000
    return stcount

def areafire(res,df):#for BA
    stcount=count_init()
    for i in range(len(res)):
        if res[i]['admin1'] in totarea.keys():
            stcount[res[i]['admin1']]+=df.BA.iloc[i]/totarea[res[i]['admin1']]*10000
    return stcount

In [6]:
#declare a class to avoid 'DataFrame' object has no attribute 'dtype' errors, and overwriting
class states:
    def __init__(self):
        self.states=gpd.read_file("../Data/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")
        
    def add_column(self,colnum,colname,col):
        self.states.insert(colnum,colname,col,True)
        
    def delete_column(self,colname):
        self.states=self.states.drop(columns=[colname])

In [7]:
#code inspired by: 
#- https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9
#- https://stackoverflow.com/questions/55362916/how-to-pass-the-slider-value-in-bokeh-back-to-python-code
#- https://stackoverflow.com/questions/56343933/bokeh-second-legend-showing-scatter-radius

#initialise s of object states

s=states()

#Sliders:
#    - year: year of the plot
#    - month: month of the plot
#    - opacity: Opacity of the scatter circles
#    - sCNTBA: choice of the mode for color scale (CNT:0, BA:1)
#    - lCNTBA: choice of the mode for scatter plot (CNT:0, BA:1)
def update(year,month,opacity,sCNTBA,lCNTBA):
    extract=df1[ (df1["year"]==year) & (df1["month"]==month) ]

    if sCNTBA==1:
        lba=[]
        lba1=[]
        ba=areafire(res,extract)
        dba=dict({'States':list(ba.keys()),'Number':list(ba.values())})
        for i in range(len(s.states)):
            if s.states["NAME"].iloc[i] in dba["States"]:
                lba.append(math.log(ba[s.states["NAME"].iloc[i]]+1))
                lba1.append(ba[s.states["NAME"].iloc[i]])
            else:
                lba.append(0)
                lba1.append(0)
        s.add_column(2,"logba",lba)
        s.add_column(2,"ba",lba1)
        geosource = GeoJSONDataSource(geojson = s.states.to_json())
        s.delete_column("logba")
        s.delete_column("ba")
    
    else:
        lcf=[]
        lcf1=[]
        cf=countfire(res,extract)
        dcf=dict({'States':list(cf.keys()),'Number':list(cf.values())})
        for i in range(len(s.states)):
            if s.states["NAME"].iloc[i] in dcf["States"]:
                lcf1.append(cf[s.states["NAME"].iloc[i]])
                lcf.append(math.log(cf[s.states["NAME"].iloc[i]]+1))
            else:
                lcf1.append(0)
                lcf.append(0)
        s.add_column(2,"logcf",lcf)
        s.add_column(2,"cf",lcf1)
        geosource = GeoJSONDataSource(geojson = s.states.to_json())
        s.delete_column("logcf")
        s.delete_column("cf") 

    palette = brewer['YlGnBu'][8]
    palette = palette[::-1]

    if sCNTBA==0:
        color_mapper = LinearColorMapper(palette = palette, low = 0, high = 8)
        tick_labels = {"0": "2⁰", "1": "2¹",
         "2":"2²", "3":"2³",
         "4":"2⁴", "5":"2⁵",
         "6":"2⁶", "7":"2⁷",
         "8":"2⁸+"}
    else:
        color_mapper = LinearColorMapper(palette = palette, low = 0, high = 8)
        tick_labels = {"0": "2⁰", "1": "2¹",
         "2":"2²", "3":"2³",
         "4":"2⁴", "5":"2⁵",
         "6":"2⁶", "7":"2⁷",
         "8":"2⁸+"}

    color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = "horizontal",
                    major_label_overrides = tick_labels)
    
    if sCNTBA==0:
        a="Number of wildfires"
    else:
        a="Burnt area from wildfires"

    p = figure(title = a+" in the United States in 0"+str(int(month))+'/'+str(int(year)), 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset, save")
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.x_range=Range1d(-127, -65)
    p.y_range=Range1d(23, 50)

    if sCNTBA==0:
        states1 = p.patches('xs','ys', source = geosource,
                   fill_color = {"field" :'logcf',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)
    else:
        states1 = p.patches('xs','ys', source = geosource,
                   fill_color = {"field" :'logba',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

    if sCNTBA==0:
        p.add_tools(HoverTool(renderers = [states1],
                      tooltips = [('State','@NAME'),
                                ('Number of Wildfires per 10⁵km²','@cf'),
                                ]))
    else:
        p.add_tools(HoverTool(renderers = [states1],
                      tooltips = [('State','@NAME'),
                                ('Burnt Area(acre) per 10⁴km²','@ba'),
                                ]))
        
    if opacity>0:
        if lCNTBA==0:
            extract=extract[extract["CNT"]!=0]
            extract["logCNT"]=[math.log(i+1)*4 for i in extract['CNT']]
            cdsextract=ColumnDataSource(data={"x":extract["lon"],"y":extract['lat'],"BA":extract['BA'],"CNT":extract["CNT"],"logCNT":extract["logCNT"]})
            sites=p.circle(x='x',y='y',source=cdsextract, color = 'red', size = "logCNT",alpha = opacity)
            p.add_tools(HoverTool(renderers = [sites],
                          tooltips = [('Number of Wildfires', '@CNT'),
                                      ('Burnt Area(acre)','@BA'),
                                      ('Latitude', '@y'),
                                     ('Longitude','@x')]))
    
            event_radius_dummy_1 = p.circle(1,1,radius=0,fill_alpha=1.0, line_color='red', color='red',name='event_radius_dummy_1')

            event_legend1 = Legend(items=[
            LegendItem(label='2⁰', renderers=[event_radius_dummy_1])],
            location=(850,10), label_standoff=8, label_height=3)

            event_legend2 = Legend(items=[
            LegendItem(label='2¹', renderers=[event_radius_dummy_1])],
            location=(848,30), label_standoff=5)

            event_legend3 = Legend(items=[
            LegendItem(label='2²', renderers=[event_radius_dummy_1])],
            location=(846,55), label_standoff=3)

            event_legend4 = Legend(items=[
            LegendItem(label='2⁴', renderers=[event_radius_dummy_1])],
            location=(840,85), label_standoff=0)

            event_legend_list = [event_legend1,event_legend2,event_legend3,event_legend4]
            for legend in event_legend_list:
                p.add_layout(legend)

            size_list = [2,8,14,25]
            index_list = [1,2,3,4]

            for index, size in zip(index_list, size_list):
                p.legend[index-1].glyph_height = size
                p.legend[index-1].glyph_width = size
                p.legend[index-1].padding = 0
                p.legend[index-1].margin = 0
                p.legend[index-1].border_line_alpha = 0
                p.legend[index-1].background_fill_alpha = 0

        else:
            extract=extract[extract["BA"]!=0]
            extract["logBA"]=[math.log(i+1)*2 for i in extract['BA']]
            cdsextract=ColumnDataSource(data={"x":extract["lon"],"y":extract['lat'],"BA":extract['BA'],"CNT":extract["CNT"],"logBA":extract["logBA"]})
            sites=p.circle(x='x',y='y',source=cdsextract,color = 'red', size = 'logBA',alpha = opacity)
            event_radius_dummy_1 = p.circle(1,1,radius=0,fill_alpha=1.0, line_color='red', color='red',name='event_radius_dummy_1')

            event_legend1 = Legend(items=[
            LegendItem(label='2¹', renderers=[event_radius_dummy_1])],
            location=(850,10), label_standoff=8, label_height=3)

            event_legend2 = Legend(items=[
            LegendItem(label='2⁴', renderers=[event_radius_dummy_1])],
            location=(848,30), label_standoff=5)

            event_legend3 = Legend(items=[
            LegendItem(label='2⁶', renderers=[event_radius_dummy_1])],
            location=(846,55), label_standoff=3)

            event_legend4 = Legend(items=[
            LegendItem(label='2¹⁰', renderers=[event_radius_dummy_1])],
            location=(840,85), label_standoff=0)

            event_legend_list = [event_legend1,event_legend2,event_legend3,event_legend4]
            for legend in event_legend_list:
                p.add_layout(legend)

            size_list = [2,8,14,25]
            index_list = [1,2,3,4]

            for index, size in zip(index_list, size_list):
                p.legend[index-1].glyph_height = size
                p.legend[index-1].glyph_width = size
                p.legend[index-1].padding = 0
                p.legend[index-1].margin = 0
                p.legend[index-1].border_line_alpha = 0
                p.legend[index-1].background_fill_alpha = 0
        

    p.add_layout(color_bar, "below")
    output_notebook()
    show(p,notebook_handle=True)
    
    push_notebook()

_ = interact(update, year =FloatSlider(min=1993, max=2015,
                                     step=1, value= 1993),
             month=FloatSlider(min=3, max=9, step=1, value= 3),
             opacity=FloatSlider(min=0, max=0.5, step=.1, value= .5),
             sCNTBA=FloatSlider(min=0,max=1,step=1,value=0),
             lCNTBA=FloatSlider(min=0,max=1,step=1,value=0))


interactive(children=(FloatSlider(value=1993.0, description='year', max=2015.0, min=1993.0, step=1.0), FloatSl…

In [8]:
dfstate=pd.DataFrame()
st=count_init()
#dfstate["year"]
#dfstate["month"]
dfstate["state"]=list(st.keys())
dfstate["month"]=np.arange(51)
#dfstate["BA"]
#dfstate["CNT"]

In [9]:
dftemp1=pd.DataFrame()
for i in range(1993,2016):
    for j in range(3,10):
        dftemp=pd.DataFrame()
        dfy=df1[df1["year"]==i]
        dfm=dfy[dfy["month"]==j]
        cnt=countfire(res,dfm)
        baa=areafire(res,dfm)
        dftemp["state"]=list(st.keys())
        dftemp["CNT"]=list(cnt.values())
        dftemp["BA"]=list(baa.values())
        dftemp["month"]=np.tile(np.array([j]),51)
        dftemp["year"]=np.tile(np.array([i]),51)
        dftemp1=dftemp1.append(dftemp,ignore_index=True,verify_integrity=True,sort=True)

In [10]:
dftemp1

,BA,CNT,month,state,year
0,211.469650,41.983693,3,Alabama,1993
1,0.000000,0.000000,3,Alaska,1993
2,27.459574,18.290576,3,Arizona,1993
3,80.976098,31.220051,3,Arkansas,1993
4,28.499860,15.567249,3,California,1993
...,...,...,...,...,...
8206,12.777672,31.592154,9,Virginia,2015
8207,114.275347,54.694819,9,Washington,2015
8208,1.019823,15.934731,9,West Virginia,2015
8209,0.185693,10.611018,9,Wisconsin,2015


In [14]:
dftemp1.to_csv("BA_CNT_per_state1.csv",index=False)

In [15]:
test=pd.read_csv("BA_CNT_per_state1.csv")

In [16]:
test

,BA,CNT,month,state,year
0,211.469650,41.983693,3,Alabama,1993
1,0.000000,0.000000,3,Alaska,1993
2,27.459574,18.290576,3,Arizona,1993
3,80.976098,31.220051,3,Arkansas,1993
4,28.499860,15.567249,3,California,1993
...,...,...,...,...,...
8206,12.777672,31.592154,9,Virginia,2015
8207,114.275347,54.694819,9,Washington,2015
8208,1.019823,15.934731,9,West Virginia,2015
8209,0.185693,10.611018,9,Wisconsin,2015


In [17]:
df1

,CNT,BA,lon,lat,area,year,month,lc1,lc2,lc3,...,clim1,clim2,clim3,clim4,clim5,clim6,clim7,clim8,clim9,clim10
0,0.0,0.000000,-95.25,49.25,0.24,1993,3,0.000006,0.015857,0.000023,...,0.222032,0.166899,265.457680,268.867126,-0.005898,9.187450e+06,-5231370.50,97849.906250,-0.000340,0.000448
1,0.0,0.000000,-94.75,49.25,0.39,1993,3,0.000005,0.002749,0.000002,...,0.184119,0.142053,265.521764,268.412354,-0.001131,6.993830e+06,-4851900.00,97954.703125,-0.000264,0.000462
2,0.0,0.000000,-122.75,48.75,0.48,1993,3,0.002420,0.103964,0.003870,...,-0.952649,0.856508,276.699820,280.594666,-0.010519,1.052566e+07,-4860741.00,100808.468750,-0.001945,0.004545
3,3.0,8.000000,-122.25,48.75,1.00,1993,3,0.002988,0.237442,0.004040,...,-0.845677,0.462569,274.943327,278.574371,-0.008420,9.359787e+06,-4653411.50,98474.648438,-0.001256,0.006174
4,0.0,0.000000,-121.75,48.75,1.00,1993,3,0.000000,0.004782,0.000196,...,-0.504174,0.195447,271.235317,274.578064,-0.005976,7.479946e+06,-3889238.75,91660.625000,-0.000502,0.008110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563978,6.0,134.600006,-80.75,25.75,1.00,2015,9,0.014934,0.017630,0.001066,...,-0.464231,0.159478,296.832658,300.201447,-0.006076,1.632964e+07,-3884615.75,101323.140625,-0.004034,0.006132
563979,1.0,30.000000,-80.25,25.75,0.66,2015,9,0.014798,0.014336,0.001253,...,-0.649578,0.404771,297.139517,300.414673,-0.005535,1.552289e+07,-3794893.75,101300.398438,-0.003913,0.006199
563980,0.0,0.000000,-81.25,25.25,0.28,2015,9,0.000000,0.000000,0.000000,...,-0.849243,0.676130,297.105902,301.003082,-0.007343,1.738194e+07,-3929039.00,101333.078125,-0.004233,0.003420
563981,2.0,179.100006,-80.75,25.25,0.76,2015,9,0.015875,0.039337,0.000373,...,-0.838257,0.589142,297.059054,300.905426,-0.006938,1.732889e+07,-4057722.00,101332.671875,-0.004125,0.003929


In [66]:
np.min(df1["clim5"])

-0.056356653571128845

In [67]:
np.max(df1["clim5"])

0.0004823761119041592

In [68]:
np.mean(df1["clim5"])

-0.011894574192339904

In [69]:
np.quantile(df1["clim5"],[0.25,0.5,0.75])

array([-0.01591428, -0.00956685, -0.00699095])

In [41]:
len(df1["clim10"].unique())

249072

In [56]:
df1.isna().sum()

CNT         0
BA          0
lon         0
lat         0
area        0
year        0
month       0
lc1         0
lc2         0
lc3         0
lc4         0
lc5         0
lc6         0
lc7         0
lc8         0
lc9         0
lc10        0
lc11        0
lc12        0
lc13        0
lc14        0
lc15        0
lc16        0
lc17        0
lc18        0
altiMean    0
altiSD      0
clim1       0
clim2       0
clim3       0
clim4       0
clim5       0
clim6       0
clim7       0
clim8       0
clim9       0
clim10      0
dtype: int64

In [57]:
result = pyreadr.read_r('../Data/data_train_DF.RData') 
dfwithna = result["data_train_DF"]

In [63]:
dfwithna[dfwithna["BA"].isna()==True]

,CNT,BA,lon,lat,area,year,month,lc1,lc2,lc3,...,clim1,clim2,clim3,clim4,clim5,clim6,clim7,clim8,clim9,clim10
24521,0.0,NaN,-95.25,49.25,0.24,1994,3,5.753801e-06,0.015857,0.000023,...,0.946406,0.144371,265.391803,270.979340,-0.009633,9.305138e+06,-5300118.0,97331.609375,-0.000455,0.000782
24522,0.0,NaN,-94.75,49.25,0.39,1994,3,5.032750e-06,0.002748,0.000002,...,1.045387,0.154082,266.060096,270.119049,-0.001791,6.697112e+06,-4841393.5,97431.398438,-0.000468,0.000770
24539,0.0,NaN,-114.75,48.75,1.00,1994,3,4.456347e-07,0.017078,0.000004,...,0.447794,0.551841,267.112260,271.830109,-0.008446,9.470027e+06,-5538610.0,85556.843750,-0.000299,0.001417
24540,0.0,NaN,-114.25,48.75,1.00,1994,3,8.880403e-05,0.008246,0.000005,...,0.639550,0.540169,266.982194,271.598053,-0.008254,9.411668e+06,-5585204.5,84947.179688,-0.000287,0.002026
24541,0.0,NaN,-113.75,48.75,1.00,1994,3,2.259169e-03,0.007954,0.000680,...,1.074054,0.833835,266.063014,270.877045,-0.008172,9.708764e+06,-5690692.5,82714.500000,-0.000364,0.001874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539453,NaN,NaN,-80.25,26.25,0.80,2014,9,1.564121e-02,0.092375,0.001888,...,-1.401931,0.493013,296.177424,299.561981,-0.005558,1.503111e+07,-3750171.0,101506.929688,-0.003868,0.007151
539455,0.0,NaN,-97.25,25.75,0.20,2014,9,2.407331e-02,0.155059,0.005064,...,-2.755768,0.709710,296.900647,300.315582,-0.005744,1.457064e+07,-3652361.5,101261.296875,-0.003552,0.007080
539457,0.0,NaN,-80.75,25.75,1.00,2014,9,1.494695e-02,0.017654,0.001066,...,-1.304978,0.109365,295.976929,299.691437,-0.006194,1.599233e+07,-3920195.0,101504.890625,-0.004009,0.006439
539459,NaN,NaN,-81.25,25.25,0.28,2014,9,0.000000e+00,0.000000,0.000000,...,-2.374217,0.536569,296.145054,300.574768,-0.007897,1.674244e+07,-3991987.5,101507.390625,-0.004056,0.003803


In [64]:
dfwithna[dfwithna["CNT"].isna()==True]

,CNT,BA,lon,lat,area,year,month,lc1,lc2,lc3,...,clim1,clim2,clim3,clim4,clim5,clim6,clim7,clim8,clim9,clim10
24523,NaN,0.000000,-122.75,48.75,0.48,1994,3,2.392067e-03,0.103736,0.003870,...,0.011068,0.968480,276.763425,280.494415,-0.010566,1.171439e+07,-5558195.50,100849.781250,-0.002340,0.004444
24530,NaN,147.000000,-119.25,48.75,1.00,1994,3,1.300255e-02,0.035858,0.015668,...,-0.095432,0.339641,269.058750,274.362122,-0.005310,8.828294e+06,-5754396.50,90521.390625,-0.000236,0.000691
24531,NaN,0.000000,-118.75,48.75,1.00,1994,3,1.422123e-03,0.006885,0.000766,...,0.230673,0.496038,268.924607,273.778320,-0.009975,1.032284e+07,-5866697.00,89332.390625,-0.000267,0.000894
24532,NaN,85.099998,-118.25,48.75,1.00,1994,3,5.722631e-08,0.018547,0.000019,...,0.474621,0.419521,268.860593,273.596710,-0.008883,9.872180e+06,-5464664.50,89211.289062,-0.000315,0.000931
24533,NaN,40.000000,-117.75,48.75,1.00,1994,3,1.271155e-06,0.030635,0.000080,...,0.019821,0.309398,269.392106,274.567444,-0.009136,9.871210e+06,-5563295.50,90648.484375,-0.000304,0.001098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539426,NaN,0.000000,-98.25,27.25,1.00,2014,9,3.749319e-01,0.070679,0.090726,...,-2.230624,0.297846,295.265714,299.885956,-0.006544,1.382598e+07,-4045789.25,100440.687500,-0.002538,0.005058
539436,NaN,0.000000,-98.75,26.75,1.00,2014,9,3.077783e-01,0.075341,0.062105,...,-2.708913,0.674322,294.927894,300.079376,-0.006399,1.391755e+07,-4233384.50,99857.601562,-0.002500,0.004229
539437,NaN,0.000000,-98.25,26.75,1.00,2014,9,3.381913e-01,0.147778,0.097225,...,-2.310834,0.300896,295.623817,299.955597,-0.006019,1.376171e+07,-3904534.50,100616.070312,-0.002782,0.005123
539453,NaN,NaN,-80.25,26.25,0.80,2014,9,1.564121e-02,0.092375,0.001888,...,-1.401931,0.493013,296.177424,299.561981,-0.005558,1.503111e+07,-3750171.00,101506.929688,-0.003868,0.007151


In [65]:
np.max(df1["BA"])

538054.375